In [1]:
import networkx as nx
import random as rd
import numpy as np
import pandas as pd

alpha = 1 # threeshold for passing message
omega = 1 #thresgold for stance flip
class agent:
    def __init__(self, i, vulunerability = True, stance = 1):
        self.stance = stance # 1 is pro, -1 is anti
        self.vulunerable = vulunerability
        self.index = i
        self.associated_msg = [] #list of index for associated message 

    def stance_flip(self):
        self.stance = self.stance * (-1)

    def get_msg(self, new_msg):
        self.associated_msg.append(new_msg)

    def purge_msg(self):
        new_msg_list = []
        for msg in self.associated_msg:
            if msg.freshness > 0:
                new_msg_list.append(msg)
        new_msg_no_rep = list(set(new_msg_list))  
        #currently does not purge repeated msg because one can see multiple RT of same msg
        #PURGE repeated msg for practical reason, otherwise msg explode
        self.associated_msg = new_msg_no_rep

    def calculate_msg(self):
        SE, SL, DE, DL = [0,0,0,0]
        if len(self.associated_msg) > 0:
            for message in self.associated_msg:
                if message.freshness > 0:
                    if message.stance == self.stance and message.manuver == 'E':
                        SE += 1
                    elif message.stance == self.stance and message.manuver == 'L':
                        SL += 1
                    elif message.stance != self.stance and message.manuver == 'E':
                        DE += 1
                    elif message.stance != self.stance and message.manuver == 'L':
                        DL += 1
        return(SE, SL, DE, DL)


In [2]:
class msg:
    def __init__(self,i, manuver, stance = 1):  # 1 is pro, -1 is anti
        self.index = i
        self.stance = stance
        self.manuver = manuver # E - emotional L - logical
        self.freshness = 3 # start from 3

    def msg_age(self):
        if self.freshness >0:
            self.freshness -= 1

In [3]:
def news_generate(network, agent_dict, msg_list, news_agent_index, message_index, manuver, stance):
    #generate new message
    # i is the index for 
    new_msg = msg(message_index,manuver, stance)

    # add msg to list to keep track of aging of msg
    msg_list.append(new_msg)

    #get neighbor
    neighbors = [n for n in network.neighbors(news_agent_index)] #list of index for neighbors
    for j in neighbors:
        agent = agent_dict.get(j)
        agent.get_msg(new_msg)

In [4]:
alpha = 2
omega = 2

def singe_agent_spread(network,agent_dict, agent, alpha = alpha, omega = omega):
    SE, SL, DE, DL = agent.calculate_msg()
    neighbors = [n for n in network.neighbors(agent.index)]
    threshold = 0 
    flip_thresold = 0 
    if agent.vulunerable == False:
        threshold = SE + SL - (DE + DL)
        flip_thresold = (DE + DL) - (SE + SL)
    else:
        threshold = 2*SE + SL - (0*DE + DL) 
        flip_thresold =  (0*DE + DL) - (2*SE + SL)
    
    print('agent',agent.index ,'threshold:', threshold, ', flip:', flip_thresold)
    if threshold > alpha and len(agent.associated_msg)>0:
        print('spread')
        for msg  in agent.associated_msg:
            #spread only new fresh msg that have the samce stance
            if msg.freshness > 0 and msg.stance == agent.stance:
                print('msg ' + str(msg.index) + ' frensh and spreading')
                for j in neighbors:
                    msg_receiver = agent_dict[j]
                    print(j, msg_receiver) 
                    if j > 1:
                    #type(agent_dict[j]) == type(agent): 
                        """ THIS IS WIRED PROBLEM """
                        print(' to agent ' + str(msg_receiver.index))
                        agent_dict[j].get_msg(msg)
                        

    if flip_thresold > omega:
        print('flip')
        agent.stance_flip()

def age_msg(msg_list):
    for msg in msg_list:
        msg.msg_age()

def purge_all_agent_msg(agent_dict):
    for i in agent_dict:
        if i > 1:
        #type(agent_dict[i]) == agent:
            agent_dict[i].purge_msg()
    

In [5]:
def intialize_agent(agent_num, pct_vul, p=0.4, seed = 42):
    """
    agent_num: numbers of agent
    pct_vul: percentage of vulunerable agents in fraction (e.g 0.5)
    p: probability of a edge between two nodes - for network
    seed: random seed - for network
    not coded - agent stance 50% pro 50% against
    """
    G =  nx.erdos_renyi_graph(agent_num, p, seed=seed, directed=False)
    agent_dict = {}
    #default first two agent to be news
    agent_dict.update({0: 'news_0, +1'}) 
    agent_dict.update({1: 'news_1, -1'})

    for index in range(2,agent_num):
        vul = (rd.random() < pct_vul)
        stc = rd.choice([-1,1])
        new_agent = agent(i= index, vulunerability = vul, stance = stc)
        agent_dict.update({index:new_agent})

    return(G, agent_dict)

def initialize_agent_data(agent_num):
    """
    initialize the data collection, output two empty pandas df with index
    """
    index_list = [n for n in range(agent_num)]
    df_stance = pd.DataFrame(columns = index_list )
    df_msgNum = pd.DataFrame(columns = index_list )
    return(df_stance, df_msgNum)

def agent_snap_shot(agent_dict):
    """ 
    input agent_dict, output two list of agent's current stance and number of msh
    can implement to present content of such msg too
    """
    stance_list= []
    associated_msg_len_list = []
    for i in agent_dict:
        if i == 0:
            stance_list.append(1)
            associated_msg_len_list.append(np.NaN)
        elif i == 1:
            stance_list.append(-1)
            associated_msg_len_list.append(np.NaN)
        else:
            stance_list.append(agent_dict[i].stance)
            associated_msg_len_list.append(len(agent_dict[i].associated_msg))

    return(stance_list,associated_msg_len_list)

def update_df(agent_dict, df_stance, df_msg):
    """ 
    take global variable agent_dict, df_stance, df_msg
    add a row after each time step
    """
    stance_l, msg_l_l = agent_snap_shot(agent_dict)
    df_stance.loc[len(df_stance)] = stance_l
    df_msg.loc[len(df_stance)] = msg_l_l


def time_step(G, agent_dict,msg_list, msg_index, manuver_pct_E, df_stance, df_msg):
    """
    G: networkX network object
    agent_dict: dictionary of agent, first twi (0,1) is news agent
    msg_list: global variable of message, used for aging msg
    msg_index: index for the new msg, should be i in some loop
    manuver_pct_E: percent chance of a msg using Emotion as focus
    """
    #generate news
    stance = lambda pct: 'E' if (rd.random() <pct) else 'L' 
    stance_news_0 = stance(manuver_pct_E)
    stance_news_1 = stance(manuver_pct_E)
    news_generate(G,agent_dict, msg_list, news_agent_index = 0, message_index = msg_index, manuver = stance_news_0, stance = 1)
    news_generate(G,agent_dict, msg_list, news_agent_index = 1, message_index = msg_index, manuver = stance_news_1, stance = -1)

    #then make agent spread the news
    for i in agent_dict:
        if i>1:
            singe_agent_spread(G, agent_dict, agent_dict[i])

    #age the msg and purge agent associated msg list
    age_msg(msg_list)
    purge_all_agent_msg(agent_dict)

    #take snapshot
    update_df(agent_dict, df_stance, df_msg)




## Test scenario

In [ ]:
######## 1 - testing agent ###########
#initialize agent
agent_1 = agent(i=1, vulunerability = False, stance =  -1 )

#initialize msg
msg_1 = msg(1, 'E', stance = 1)
msg_2 = msg(2, 'L', stance = 1)

#agent get message
agent_1.get_msg(msg_2)

#message update (lose freshness)
msg_1.msg_update()

#examine msg freshness
msg_1.freshness

#purge old message (externalized memory)
agent_1.purge_msg()

#check associated msg
agent_1.associated_msg

In [ ]:
########## 2 - testing new generation function ##########

import networkx as nx
#create empty graph
#G = nx.Graph()

#initialize a randome graph
G = nx.erdos_renyi_graph(6, 0.4, seed=42, directed=False)
msg_list = []
#initialize agent
agent_dict = {}
agent_dict.update({0: 'news_0'})
agent_dict.update({1: 'news_1'})

#initialize agent
agent_2 = agent(i=2, vulunerability = False, stance =  -1 )
agent_3 = agent(i=3, vulunerability = False, stance =  -1 )
agent_4 = agent(i=4, vulunerability = True, stance =  1 )
agent_5 = agent(i=5, vulunerability = False, stance =  1 )

agent_dict.update({2:agent_2})
agent_dict.update({3:agent_3})
agent_dict.update({4:agent_4})
agent_dict.update({5:agent_5})

# generate msg
news_generate(G,agent_dict, msg_list, news_agent_index = 1, message_index = 1, manuver = 'E', stance = 1)

#print agent_dict to make sure all agent type correct
#agent_dict
#print graph for debugging
pos = nx.spring_layout(G)
nx.draw(G,with_labels = True)

#examnie associated msg
for i in range(2,6):
    print(agent_dict.get(i).associated_msg)

In [ ]:
##### 3 - test single agent_spread age_msg and purge_all_msg ########3


for i in agent_dict:
    if i>1:
        singe_agent_spread(G, agent_dict, agent_dict[i])

age_msg(msg_list)
purge_all_agent_msg(agent_dict)

for i in range(2,6):
    print((agent_dict.get(i).index, agent_dict.get(i).associated_msg))

#generate another msg
news_generate(G,agent_dict, msg_list, news_agent_index = 0, message_index = 2, manuver = 'E', stance = -1)


for i in agent_dict:
    if i>1:
        singe_agent_spread(G, agent_dict, agent_dict[i])

age_msg(msg_list)
purge_all_agent_msg(agent_dict)

for i in range(2,6):
    print((agent_dict.get(i).index, agent_dict.get(i).associated_msg))

In [ ]:
#news agent take the index of 1 and 2
### fix in later version for zero indexing issue, take 0 and 1
#### Appendix 1 - testing new usage of networkX of which to use agent directly as node ########
import networkx as nx
G = nx.Graph()
#initialize agent
agent_3 = agent(i=3, vulunerability = False, stance =  -1 )
agent_4 = agent(i=4, vulunerability = False, stance =  -1 )
agent_5 = agent(i=5, vulunerability = True, stance =  1 )
agent_6 = agent(i=6, vulunerability = False, stance =  1 )

#agent_list = [(agent_1.index, agent_1), (agent_2.index,agent_2), (agent_3.index,agent_3), (agent_4.index,agent_4)]
agent_list = [agent_5, agent_6, agent_3, agent_4]

G.add_nodes_from(agent_list)

G.nodes




In [ ]:
###### Appendix 2 debugging the singe_agent_spread function ##############
##### MAIN ISSUE is the 'type(agent_4) is agent' or 'type(agent)' problem, look like a systematic bug 
msg = agent_4.associated_msg[0]
msg = agent_4.associated_msg[0]

for j in neighbors:
    print(agent_dict[j])
    if type(agent_dict[j]) is agent:
        print((agent_dict[j].index, agent_dict[j].stance))
        agent_dict[j].get_msg(msg)

for msg  in agent_4.associated_msg:
    print(msg)

update(G, agent_dict, agent_4)

for i in range(2,6):
    print((agent_dict.get(i).index, agent_dict.get(i).associated_msg))

In [6]:
####### 4 - test runing of one set
msg_list=[]
G, agent_dict = intialize_agent(agent_num = 20, pct_vul = 0.2, p=0.4, seed = 42)
df_stance, df_msgNum = initialize_agent_data(20)
time_step(G, agent_dict,msg_list, 0, 0.5, df_stance, df_msgNum)


agent 2 threshold: 0 , flip: 0
agent 3 threshold: 1 , flip: -1
agent 4 threshold: -1 , flip: 1
agent 5 threshold: -1 , flip: 1
agent 6 threshold: 1 , flip: -1
agent 7 threshold: 0 , flip: 0
agent 8 threshold: 0 , flip: 0
agent 9 threshold: -1 , flip: 1
agent 10 threshold: 0 , flip: 0
agent 11 threshold: 1 , flip: -1
agent 12 threshold: 0 , flip: 0
agent 13 threshold: -1 , flip: 1
agent 14 threshold: 1 , flip: -1
agent 15 threshold: 0 , flip: 0
agent 16 threshold: 0 , flip: 0
agent 17 threshold: 0 , flip: 0
agent 18 threshold: 0 , flip: 0
agent 19 threshold: 0 , flip: 0


In [7]:
time_step(G, agent_dict,msg_list, 1, 0.5, df_stance, df_msgNum)

agent 2 threshold: 1 , flip: -1
agent 3 threshold: 2 , flip: -2
agent 4 threshold: -2 , flip: 2
agent 5 threshold: -2 , flip: 2
agent 6 threshold: 2 , flip: -2
agent 7 threshold: 0 , flip: 0
agent 8 threshold: 0 , flip: 0
agent 9 threshold: -2 , flip: 2
agent 10 threshold: 0 , flip: 0
agent 11 threshold: 2 , flip: -2
agent 12 threshold: 0 , flip: 0
agent 13 threshold: -1 , flip: 1
agent 14 threshold: 2 , flip: -2
agent 15 threshold: 0 , flip: 0
agent 16 threshold: 0 , flip: 0
agent 17 threshold: 0 , flip: 0
agent 18 threshold: 0 , flip: 0
agent 19 threshold: 0 , flip: 0


In [8]:
time_step(G, agent_dict,msg_list, 2, 0.5, df_stance, df_msgNum)

agent 2 threshold: 1 , flip: -1
agent 3 threshold: 3 , flip: -3
spread
msg 0 frensh and spreading
0 news_0, +1
6 <__main__.agent object at 0x7ff2b0b59d30>
 to agent 6
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
16 <__main__.agent object at 0x7ff2d0496130>
 to agent 16
17 <__main__.agent object at 0x7ff2d0496610>
 to agent 17
18 <__main__.agent object at 0x7ff2d0496400>
 to agent 18
19 <__main__.agent object at 0x7ff2d0496880>
 to agent 19
msg 1 frensh and spreading
0 news_0, +1
6 <__main__.agent object at 0x7ff2b0b59d30>
 to agent 6
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
16 <__main__.agent object at 0x7ff2d0496130>
 to agent 16
17 <__main__.agent object at 0x7ff2d0496610>
 to agent 17
18 <__main__.agent object at 0x7ff2d0496400>
 to agent 18
19 <__main__.agent object at 0x7ff2d0496880>
 to agent 19
msg 2 frensh and spreading
0 news_0, +1
6 <__main

In [9]:
time_step(G, agent_dict,msg_list, 3, 0.5, df_stance, df_msgNum)

agent 2 threshold: 1 , flip: -1
agent 3 threshold: 3 , flip: -3
spread
msg 2 frensh and spreading
0 news_0, +1
6 <__main__.agent object at 0x7ff2b0b59d30>
 to agent 6
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
16 <__main__.agent object at 0x7ff2d0496130>
 to agent 16
17 <__main__.agent object at 0x7ff2d0496610>
 to agent 17
18 <__main__.agent object at 0x7ff2d0496400>
 to agent 18
19 <__main__.agent object at 0x7ff2d0496880>
 to agent 19
msg 1 frensh and spreading
0 news_0, +1
6 <__main__.agent object at 0x7ff2b0b59d30>
 to agent 6
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
16 <__main__.agent object at 0x7ff2d0496130>
 to agent 16
17 <__main__.agent object at 0x7ff2d0496610>
 to agent 17
18 <__main__.agent object at 0x7ff2d0496400>
 to agent 18
19 <__main__.agent object at 0x7ff2d0496880>
 to agent 19
msg 3 frensh and spreading
0 news_0, +1
6 <__main

In [10]:
time_step(G, agent_dict,msg_list, 4, 0.5, df_stance, df_msgNum)

agent 2 threshold: 1 , flip: -1
agent 3 threshold: 3 , flip: -3
spread
msg 3 frensh and spreading
0 news_0, +1
6 <__main__.agent object at 0x7ff2b0b59d30>
 to agent 6
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
16 <__main__.agent object at 0x7ff2d0496130>
 to agent 16
17 <__main__.agent object at 0x7ff2d0496610>
 to agent 17
18 <__main__.agent object at 0x7ff2d0496400>
 to agent 18
19 <__main__.agent object at 0x7ff2d0496880>
 to agent 19
msg 2 frensh and spreading
0 news_0, +1
6 <__main__.agent object at 0x7ff2b0b59d30>
 to agent 6
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
16 <__main__.agent object at 0x7ff2d0496130>
 to agent 16
17 <__main__.agent object at 0x7ff2d0496610>
 to agent 17
18 <__main__.agent object at 0x7ff2d0496400>
 to agent 18
19 <__main__.agent object at 0x7ff2d0496880>
 to agent 19
msg 4 frensh and spreading
0 news_0, +1
6 <__main

In [12]:
time_step(G, agent_dict,msg_list, 5, 0.5, df_stance, df_msgNum)

agent 2 threshold: 3 , flip: -3
spread
msg 4 frensh and spreading
0 news_0, +1
1 news_1, -1
7 <__main__.agent object at 0x7ff2b0b59a30>
 to agent 7
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
10 <__main__.agent object at 0x7ff2b0b59c10>
 to agent 10
11 <__main__.agent object at 0x7ff2b0b59cd0>
 to agent 11
12 <__main__.agent object at 0x7ff2b0b599d0>
 to agent 12
13 <__main__.agent object at 0x7ff2b0b59a90>
 to agent 13
15 <__main__.agent object at 0x7ff2b0b59f10>
 to agent 15
16 <__main__.agent object at 0x7ff2d0496130>
 to agent 16
17 <__main__.agent object at 0x7ff2d0496610>
 to agent 17
18 <__main__.agent object at 0x7ff2d0496400>
 to agent 18
msg 3 frensh and spreading
0 news_0, +1
1 news_1, -1
7 <__main__.agent object at 0x7ff2b0b59a30>
 to agent 7
8 <__main__.agent object at 0x7ff2b0b59e50>
 to agent 8
9 <__main__.agent object at 0x7ff2b0b59ca0>
 to agent 9
10 <__main__.agent object at 0x7ff2b0b59c10>
 to agent 

In [13]:
df_msgNum

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,NaN,NaN,2.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0
2,NaN,NaN,4.0,2.0,2.0,2.0,2.0,0.0,4.0,2.0,4.0,2.0,0.0,2.0,2.0,0.0,0.0,4.0,0.0,0.0
3,NaN,NaN,4.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0
4,NaN,NaN,4.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0
5,NaN,NaN,4.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0
6,NaN,NaN,4.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,4.0,2.0


In [14]:
df_stance

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,-1,-1,1,-1,1,-1,-1,1,1,1,1,1,-1,1,-1,1,1,-1,1
1,1,-1,-1,1,-1,1,-1,-1,1,1,1,1,1,-1,1,-1,1,1,-1,1
2,1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
list_1

In [ ]:
set(agent_dict[4].associated_msg)

### Not yet tested

In [ ]:
df.loc[index]=list


In [ ]:

def play(network, directory, mvy = 0, mry = 0):  #default: no mood noise, no mood regression
    #play game
    ##shuffule
    node_num = network.number_of_nodes()
    index = list(range(node_num))
    rd.shuffle(index)
    
    for i in index:
        #didn't shuffle neighbors but figured won't need to shuffle twice??
        neighbors = [n for n in network.neighbors(i)]
        player = directory.get(i) #return player agent
        for j in neighbors:
            player.game(directory.get(j)) #play a interaction game and update accordingly
         
    # each network only update their mood with noise and regression ONCE after everyone done playing
    if(mry == 1):
        for i in index:
            player = directory.get(i) 
            player.mood_regression()
            
    if(mvy == 1):
        for i in index:
            player = directory.get(i) 
            player.mood_vary() 

            
    #record the mood
    mood_value = []
    for i in list(directory.values()):
        mood_value.append(i.mood)
         
    return  mood_value

def run_sim(network, di, iters=100, mvy = 0, mry = 0):
    #make a storage list 
    mood_overtime = []
    for k in range(iters):
        mood_value = play(network, di, mvy, mry)
        mood_overtime.append(mean(mood_value))
        
    return mood_overtime   